# LibCBM versus CBM-CFS3 Stand level testing

In [1]:
import os, json, math
import numpy as np
import pandas as pd
%matplotlib inline


libCBM related imports

In [2]:
from libcbmwrapper import LibCBMWrapper
import libcbmconfig
import cbmconfig
import cbm_defaults
from cbm3 import CBM3

cbm3 related imports

In [3]:
import cbm3_python_helper
cbm3_python_helper.load_cbm3_python()
import cbm3_python.simulation.projectsimulator
from cbm3_python.cbm3data import sit_helper
from cbm3_python.cbm3data import cbm3_results

In [4]:
db_path = 'C:\dev\cbm_defaults\cbm_defaults.db'
n_steps = 225

age_interval = 5 #required by cbm3
num_age_classes = 40 #required by cbm3

def get_classifier_name(id):
    return str(id)

def growth_curve(x, x_0=65, L=250, k=0.1 ):
    return (x, L/(1+math.exp(-k*(x-x_0))))
    
def create_scenario(id, admin_boundary, eco_boundary, components, events):
    return {
        "id":id,
        "admin_boundary": admin_boundary,
        "eco_boundary": eco_boundary,
        "components": components,
        "events": events
    }

def generate_scenarios(random_seed, num_cases, dbpath, ndigits):
    np.random.seed(random_seed)
    species_ref = cbm_defaults.load_species_reference(dbpath, "en-CA")
    species = [k for k,v in species_ref.items() if len(k)<50 and v["forest_type_id"] in [1,3]] #exclude species names that are too long for the project database schema
    
    spatial_units = cbm_defaults.get_spatial_unit_ids_by_admin_eco_name(dbpath, "en-CA")
    random_spus = np.random.choice([",".join(x) for x in spatial_units.keys()], num_cases)
    
    disturbance_types = cbm_defaults.get_disturbance_type_ids_by_name(dbpath, "en-CA")
    
    cases = []
    for i in range(num_cases):
        num_components = np.random.randint(1,5)
        random_species = np.random.choice(list(species), num_components)
        spu = random_spus[i].split(',')
        components = []
        for c in range(num_components):
            components.append({
                "species": random_species[c],
                "age_volume_pairs": [growth_curve(x) for x in range(0,200,age_interval)]
            })

        num_disturbances = np.random.randint(0,3)
        random_dist_types = np.random.choice(list(disturbance_types), num_disturbances)
        disturbance_events = []
        min_timestep = 0
        for d in range(num_disturbances):
            
            disturbance_events.append({
             "disturbance_type": random_dist_types[d],
             "time_step": np.random.randint(min_timestep, n_steps)
            })
            
        cases.append(create_scenario(
            id = i+1,
            admin_boundary = spu[0],
            eco_boundary = spu[1],
            components = components,
            events = disturbance_events ))
    return cases

In [12]:
cases = generate_scenarios(1, 1, db_path, 2)

#pd.DataFrame({"x": list(range(0, 200 , 5)), "y": [growth_curve(x,65,100,0.1) for x in range(0, 200 , 5)]}).groupby("x").sum().plot()
#list(range(0, 200 , 20))

In [10]:
def run_libCBM(dbpath, cases, nsteps):
    
    dllpath = r'C:\dev\LibCBM\LibCBM\x64\Debug\LibCBM.dll'

    dlldir = os.path.dirname(dllpath)
    cwd = os.getcwd()
    os.chdir(dlldir)
    dll = LibCBMWrapper(dllpath)
    os.chdir(cwd)
    
    pooldef = cbm_defaults.load_cbm_pools(dbpath)
    dll.Initialize(libcbmconfig.to_string(
        {
            "pools": pooldef,
            "flux_indicators": cbm_defaults.load_flux_indicators(dbpath)
        }))
    
    #create a single classifier/classifier value for the single growth curve
    classifiers_config = cbmconfig.classifier_config([
        cbmconfig.classifier("growth_curve", [
            cbmconfig.classifier_value(get_classifier_name(c["id"])) 
            for c in cases
        ])
    ])


    transitions_config = []
    species_reference = cbm_defaults.load_species_reference(dbpath, "en-CA")
    spatial_unit_reference = cbm_defaults.get_spatial_unit_ids_by_admin_eco_name(dbpath, "en-CA")
    disturbance_types = cbm_defaults.get_disturbance_type_ids_by_name(dbpath, "en-CA")
    curves = []
    for c in cases:
        classifier_set = [get_classifier_name(c["id"])]
        merch_volumes = []
        for component in c["components"]:
            merch_volumes.append({
                "species_id": species_reference[component["species"]]["species_id"],
                "age_volume_pairs": component["age_volume_pairs"]
            })

        curve = cbmconfig.merch_volume_curve(
            classifier_set = classifier_set,
            merch_volumes = merch_volumes)
        curves.append(curve)

    merch_volume_to_biomass_config = cbmconfig.merch_volume_to_biomass_config(
        dbpath, curves)

    dll.InitializeCBM(libcbmconfig.to_string({
        "cbm_defaults": cbm_defaults.load_cbm_parameters(dbpath),
        "merch_volume_to_biomass": merch_volume_to_biomass_config,
        "classifiers": classifiers_config["classifiers"],
        "classifier_values": classifiers_config["classifier_values"],
        "transitions": []
    }))    

    nstands = len(cases)
    age = np.zeros(nstands,dtype=np.int32)
    classifiers = np.zeros((nstands,1),dtype=np.int32)
    classifiers[:,0]=[classifiers_config["classifier_index"][0][get_classifier_name(c["id"])] for c in cases]
         
    spatial_units = np.array(
        [spatial_unit_reference[(c["admin_boundary"],c["eco_boundary"])]
            for c in cases],dtype=np.int32)
    pools = np.zeros((nstands,len(pooldef)))
    cbm3 = CBM3(dll)
    cbm3.spinup(
        pools=pools,
        classifiers=classifiers,
        inventory_age=0,
        spatial_unit=spatial_units,
        historic_disturbance_type=disturbance_types["Wildfire"],
        last_pass_disturbance_type=disturbance_types["Wildfire"],
        return_interval=125,
        min_rotations=10,
        max_rotations=30,
        delay=0,
        mean_annual_temp=None)

In [13]:
run_libCBM(db_path, cases, n_steps)

AttributeError: 'CBM3' object has no attribute 'configuration'

In [88]:
def run_CBM3(cases, age_interval, num_age_classes, nsteps):
    standard_import_tool_plugin_path=sit_helper.load_standard_import_tool_plugin()

    toolbox_path = r"C:\Program Files (x86)\Operational-Scale CBM-CFS3"

    #there is a bug fix in this version of cbm/makelist for growth incrment blips
    cbm_exe_path = r"M:\CBM Tools and Development\Builds\CBMBuilds\20190530_growth_increment_fix"

    cbm3_project_dir = os.path.join(toolbox_path, "Projects", "libcbm_stand_level_testing")
    cbm3_project_path = os.path.join(cbm3_project_dir, "libcbm_stand_level_testing.mdb")
    cbm3_results_db_path = os.path.join(cbm3_project_dir, "libcbm_stand_level_testing_results.mdb")
    config_save_path = os.path.join(cbm3_project_dir, "libcbm_stand_level_testing.json")
    sit_config = sit_helper.SITConfig(
        imported_project_path=cbm3_project_path,
        initialize_mapping=True
    )
    sit_config.data_config(
        age_class_size=age_interval,
        num_age_classes=num_age_classes,
        classifiers=["admin", "eco", "identifier", "species"])
    sit_config.set_admin_eco_mapping("admin","eco")
    sit_config.set_species_classifier("species")
    for c in cases:
        cset = [c["admin_boundary"], c["eco_boundary"], get_classifier_name(c["id"]), "Spruce"]
        sit_config.add_inventory(classifier_set=cset, area=1, age=0, unfccc_land_class=0,
                            historic_disturbance="Wildfire", last_pass_disturbance="Wildfire")
        for component in c["components"]:
            sit_config.add_yield(classifier_set=cset, 
                        leading_species_classifier_value=component["species"],
                        values=[x[1] for x in component["age_volume_pairs"]])
        for event in c["events"]:
            sit_config.add_event(
                classifier_set=cset,
                disturbance_type=event["disturbance_type"],
                time_step=event["time_step"],
                target=1,
                target_type = "Area",
                sort = "SORT_BY_SW_AGE")
    sit_config.add_event(
        classifier_set=["?","?","?","?"],
        disturbance_type="Wildfire",
        time_step=nsteps+1,
        target=1,
        target_type = "Area",
        sort = "SORT_BY_SW_AGE")
    sit_config.import_project(standard_import_tool_plugin_path, config_save_path)
    cbm3_python.simulation.projectsimulator.run(
        aidb_path=os.path.join(toolbox_path, "admin", "dbs", "ArchiveIndex_Beta_Install.mdb"), 
        project_path=cbm3_project_path, 
        toolbox_installation_dir=toolbox_path,
        cbm_exe_path=cbm_exe_path,
        results_database_path = cbm3_results_db_path)
    